In [11]:
import os
import numpy as np
import pandas as pd
from collections import Counter
from tqdm import tqdm

# Where .npz patches are saved
# patch_dir = "../json files/patches_5x5"
patch_dir = "../json files/patches_5x5_visayas"

## I. Patch Extraction Pipeline: VIIRS .npz to DataFrame

This loop processes all `.npz` files for each point-of-interest (POI) and extracts daily patch statistics for each spatial extent (1×1, 3×3, 5×5).

---

### **Algorithm Overview**

1. **Directory Traversal**
    - Iterate over each POI folder in `patch_dir`.
    - For every `.npz` file (corresponding to a date), load patch data.

2. **Patch Extraction for Each Extent**
    - For each extent (`1x1`, `3x3`, `5x5`), extract pixel-level stats for:
        - DNB observed radiance (`DNB_mean`, `DNB_stdDev`)
        - Gap-filled radiance (`Gap_mean`, `Gap_stdDev`)
        - Valid pixel count and patch size (`DNB_valid_px`, `Total_px`)
        - Percent of valid pixels (`Valid_pct`)
        - Quality flags/histograms (`HQ_hist`, `QF_hist`)

3. **Row Construction**
    - Compile all metrics into a dictionary per (POI, date, extent), append to `rows`.

4. **Output**
    - Combine all rows into a tidy DataFrame for downstream analysis and visualization.

---

**This step transforms raw, per-patch .npz data into a flat table of summary statistics, indexed by POI, date, and patch size.**

In [12]:
# Extract center, 3x3, or 5x5 pixels from 5x5 array
def extract_zone(arr, size):
    if arr is None:
        return []
    arr = np.array(arr)
    if arr.ndim != 2 or arr.shape != (5, 5):
        return []
    if size == 1:
        return [arr[2, 2]]
    elif size == 3:
        return arr[1:4, 1:4].flatten().tolist()
    elif size == 5:
        return arr.flatten().tolist()
    return []

# Clean histogram values (cast to int)
def clean_hist(arr):
    return dict(Counter([int(v) for v in arr if v is not None]))

# Compute patch stats
def summarize_patch(values):
    arr = np.array([v for v in values if v is not None])
    valid = arr[arr > 0]
    return {
        'mean': valid.mean() if valid.size else None,
        'std': valid.std() if valid.size else None,
        'valid': int((valid > 0).sum()),
        'total': int(arr.size),
        'pct': float(valid.size / arr.size * 100) if arr.size else None
    }


# Loop through all .npz files
rows = []
for poi in tqdm(os.listdir(patch_dir)):
    poi_path = os.path.join(patch_dir, poi)
    if not os.path.isdir(poi_path):
        continue  # ✅ Skip files like .DS_Store

    for filename in os.listdir(poi_path):
        if not filename.endswith(".npz"):
            continue
        date_str = filename.replace(".npz", "")
        path = os.path.join(poi_path, filename)
        try:
            data = np.load(path, allow_pickle=True)
        except:
            continue

        for size, extent in zip([1, 3, 5], ['1x1', '3x3', '5x5']):
            row = {
                'date': date_str,
                'location': poi,
                'extent': extent,
                'DNB_mean': None,
                'DNB_stdDev': None,
                'Gap_mean': None,
                'Gap_stdDev': None,
                'DNB_valid_px': None,
                'Total_px': None,
                'Valid_pct': None,
                'HQ_hist': None,
                'QF_hist': None
            }

            # DNB
            dnb = extract_zone(data.get('DNB_BRDF_Corrected_NTL'), size)
            if dnb:
                stats = summarize_patch(dnb)
                row.update({
                    'DNB_mean': stats['mean'],
                    'DNB_stdDev': stats['std'],
                    'DNB_valid_px': stats['valid'],
                    'Total_px': stats['total'],
                    'Valid_pct': stats['pct']
                })

            # Gap-filled DNB
            gap = extract_zone(data.get('Gap_Filled_DNB_BRDF_Corrected_NTL'), size)
            if gap:
                stats = summarize_patch(gap)
                row['Gap_mean'] = stats['mean']
                row['Gap_stdDev'] = stats['std']

            # HQ + QF histograms
            hq = extract_zone(data.get('Latest_High_Quality_Retrieval'), size)
            if hq:
                row['HQ_hist'] = clean_hist(hq)

            qf = extract_zone(data.get('QF_Cloud_Mask'), size)
            if qf:
                row['QF_hist'] = clean_hist(qf)

            rows.append(row)

# Convert to DataFrame
df = pd.DataFrame(rows)

100%|██████████| 10/10 [00:13<00:00,  1.36s/it]


In [13]:
# Fill missing Total_px based on extent
extent_to_total = {'1x1': 1, '3x3': 9, '5x5': 25}
df['Total_px'] = df.apply(
    lambda row: extent_to_total.get(row['extent'], np.nan) if pd.isna(row['Total_px']) else row['Total_px'],
    axis=1
)

# Fill NaN DNB_valid_px with 0
df['DNB_valid_px'] = df['DNB_valid_px'].fillna(0)

# Recompute Valid_pct everywhere: (valid/total)*100, 0 if total is 0 or valid is 0
df['Valid_pct'] = np.where(
    df['Total_px'] > 0,
    df['DNB_valid_px'] / df['Total_px'] * 100,
    0
)

df

,date,location,extent,DNB_mean,DNB_stdDev,Gap_mean,Gap_stdDev,DNB_valid_px,Total_px,Valid_pct,HQ_hist,QF_hist
0,2013-09-02,Iloilo_City,1x1,12.126097,0.000000,12.126097,0.000000,1.0,1.0,100.0,{1: 1},{34: 1}
1,2013-09-02,Iloilo_City,3x3,11.278476,4.373290,11.278476,4.373290,9.0,9.0,100.0,{1: 9},{34: 9}
2,2013-09-02,Iloilo_City,5x5,13.625078,7.111626,13.625078,7.111626,25.0,25.0,100.0,{1: 25},{34: 25}
3,2013-09-16,Iloilo_City,1x1,NaN,NaN,19.738071,0.000000,0.0,1.0,0.0,{3: 1},{738: 1}
4,2013-09-16,Iloilo_City,3x3,NaN,NaN,17.048057,5.054433,0.0,9.0,0.0,{3: 9},"{226: 1, 738: 8}"
...,...,...,...,...,...,...,...,...,...,...,...,...
10945,2013-09-27,Roxas_City,3x3,NaN,NaN,16.332638,5.532716,0.0,9.0,0.0,{14: 9},{738: 9}
10946,2013-09-27,Roxas_City,5x5,NaN,NaN,9.859332,6.560536,0.0,25.0,0.0,{14: 25},{738: 25}
10947,2013-07-22,Roxas_City,1x1,NaN,NaN,18.725666,0.000000,0.0,1.0,0.0,{21: 1},{738: 1}
10948,2013-07-22,Roxas_City,3x3,NaN,NaN,15.397524,6.830412,0.0,9.0,0.0,{21: 9},{738: 9}


In [14]:
df[(df["extent"] == "3x3") & (df["location"] == "Roxas_City")]

,date,location,extent,DNB_mean,DNB_stdDev,Gap_mean,Gap_stdDev,DNB_valid_px,Total_px,Valid_pct,HQ_hist,QF_hist
9856,2013-09-02,Roxas_City,3x3,11.835731,8.03549,15.751929,7.660340,9.0,9.0,100.0,"{0: 7, 12: 1, 27: 1}",{34: 9}
9859,2013-09-16,Roxas_City,3x3,NaN,NaN,16.332638,5.532716,0.0,9.0,0.0,{3: 9},{738: 9}
9862,2013-07-13,Roxas_City,3x3,NaN,NaN,15.397524,6.830412,0.0,9.0,0.0,{12: 9},"{738: 8, 746: 1}"
9865,2013-07-07,Roxas_City,3x3,NaN,NaN,15.397524,6.830412,0.0,9.0,0.0,{6: 9},{738: 9}
9868,2013-10-23,Roxas_City,3x3,NaN,NaN,16.126110,5.615909,0.0,9.0,0.0,"{6: 1, 5: 6, 4: 2}",{242: 9}
...,...,...,...,...,...,...,...,...,...,...,...,...
10936,2013-12-17,Roxas_City,3x3,NaN,NaN,11.941144,5.082014,0.0,9.0,0.0,"{8: 1, 1: 8}","{242: 6, 114: 2, 250: 1}"
10939,2013-10-06,Roxas_City,3x3,NaN,NaN,16.332638,5.532716,0.0,9.0,0.0,"{23: 6, 7: 3}",{738: 9}
10942,2013-10-12,Roxas_City,3x3,NaN,NaN,16.126110,5.615909,0.0,9.0,0.0,{5: 9},{738: 9}
10945,2013-09-27,Roxas_City,3x3,NaN,NaN,16.332638,5.532716,0.0,9.0,0.0,{14: 9},{738: 9}


## II. Interactive NTL Patch Time Series Dashboard (Plotly)

This dashboard visualizes VIIRS nighttime lights (NTL) statistics for multiple locations and patch sizes, enabling rapid comparison of radiance, coverage, and quality over time and space.

---

### **How it works**

- **Patch Size & Location**  
  User can select any point of interest (POI) and patch extent (`1x1`, `3x3`, `5x5`). Patch size determines the spatial footprint (1, 9, or 25 pixels per patch).

- **Layout**  
  - **Top panel:** Bar chart of valid pixel coverage (%) per day (proportion of fresh, HQ=0 pixels in the patch).
  - **Bottom panel:**  
    - Red line: Gap-filled NTL mean (±1 stddev shaded band).  
    - Blue dots: Observed NTL mean (±1 stddev error bars for per-day uncertainty).

- **Dropdowns/Buttons**  
  - Select different locations and patch sizes interactively.  
  - Default view: `3x3` patch at Tacloban_City.

- **Landfall Annotation**  
  A vertical line/annotation marks Typhoon Haiyan’s landfall for reference.

- **Visibility Control**  
  Only traces for the currently selected location and patch size are shown; all others are hidden.

---

### **Usage Notes**
- **Valid pixel %** reflects the average daily percent of pixels in the patch with high-quality (HQ=0) values.
- **Gap-filled radiance** represents interpolated or “filled” NTL for cloudy or missing days.
- **Observed radiance** is shown only when direct measurements are available.

This tool provides a compact, interactive way to analyze NTL trends, cloud coverage, and recovery at multiple scales after a disaster or for monitoring urban dynamics.

In [15]:
# Define your POI list
poi_data = [
    {'name': 'Roxas_City',  'coord': [122.7520, 11.5807]}, #11.5807954781089, 122.75200550379739
    {'name': 'Kalibo',      'coord': [122.3638, 11.7061]},
    {'name': 'Iloilo_City', 'coord': [122.5644, 10.7202]},
    {'name': 'Bogo_City',   'coord': [124.0059, 11.0513]}, #11.051268572553099, 124.005871517945
    {'name': 'Cadiz_City',  'coord': [123.3052, 10.9565]}, #10.956578136912789, 123.30525205025558
    {'name': 'Boracay',    'coord': [121.9254, 11.9613]}, #11.961286376803649, 121.92542690380587
    {'name': 'Estancia',    'coord': [123.1511, 11.4527]}, #11.45268266257498, 123.15105201024426
    {'name': 'Cebu City',  'coord': [123.9055, 10.3181]}, #10.318069584021458, 123.90550813175145
    {'name': 'Bacolod City',  'coord': [122.9480, 10.6699]}, #10.669900014787668, 122.94809590698326
    {'name': 'Puerto Princessa City',  'coord': [118.7362, 9.7392]}, #9.739162629001392, 118.73620891251826
]

In [16]:
import pandas as pd
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# --- Setup ---
extent_codes = ['1x1', '3x3', '5x5']
extent_label_map = {'1x1': '1 px radius', '3x3': '3 px radius', '5x5': '5 px radius'}

# poi_data = [
#     {'name': 'Tacloban_City'}, {'name': 'Ormoc_City'}, {'name': 'Baybay_City'},
#     {'name': 'Palo'}, {'name': 'Alang_Alang'}, {'name': 'Guiuan'},
#     {'name': 'Samar_Forest_Reserve'}, {'name': 'Mt_Nacolod_Upland'},
#     {'name': 'Central_Leyte_Forest'}, {'name': 'Southern_Leyte_Gulf'}
# ]
locations = [p['name'] for p in poi_data]

fig = make_subplots(
    rows=2, cols=1,
    row_heights=[0.20, 0.70],
    shared_xaxes=True,
    vertical_spacing=0.10,
    subplot_titles=("Observed Pixel Coverage (%)", "NTL Radiance Time Series")
)

trace_settings = []

for extent in extent_codes:
    for loc in locations:
        dfl = df[(df['extent'] == extent) & (df['location'] == loc)].copy()
        if dfl.empty:
            continue
        # --- Always make sure dates are sorted and correct type
        dfl['date'] = pd.to_datetime(dfl['date'])
        dfl = dfl.sort_values('date')

        # Coverage bar
        fig.add_trace(
            go.Bar(
                x=dfl['date'], y=dfl['Valid_pct'],
                name='Coverage', marker_color='green', opacity=1, showlegend=False
            ), row=1, col=1
        )
        trace_settings.append((extent, loc, 'coverage'))

        # Gap-Filled mean and stddev band
        dfl_gap = dfl.dropna(subset=['Gap_mean', 'Gap_stdDev'])
        gap_upper = dfl_gap['Gap_mean'] + dfl_gap['Gap_stdDev']
        gap_lower = dfl_gap['Gap_mean'] - dfl_gap['Gap_stdDev']

        # Gap mean (red line)
        fig.add_trace(
            go.Scatter(
                x=dfl_gap['date'], y=dfl_gap['Gap_mean'],
                mode='lines',
                line=dict(color='red'),
                name='Gap-Filled',
                showlegend=True if (extent == '3x3' and loc == locations[0]) else False
            ), row=2, col=1
        )
        trace_settings.append((extent, loc, 'gap'))

        # Stddev band (upper, invisible)
        fig.add_trace(
            go.Scatter(
                x=dfl_gap['date'], y=gap_upper,
                mode='lines', line=dict(width=0),
                showlegend=False, hoverinfo='skip'
            ), row=2, col=1
        )
        trace_settings.append((extent, loc, 'gap_std_upper'))

        # Stddev band (lower, fill)
        fig.add_trace(
            go.Scatter(
                x=dfl_gap['date'], y=gap_lower,
                mode='lines', line=dict(width=0),
                fill='tonexty', fillcolor='rgba(255,0,0,0.13)',
                showlegend=False, hoverinfo='skip'
            ), row=2, col=1
        )
        trace_settings.append((extent, loc, 'gap_std_lower'))

        # Observed DNB (blue dots)
        dfl_dnb = dfl.dropna(subset=['DNB_mean', 'DNB_stdDev'])
        fig.add_trace(
            go.Scatter(
                x=dfl_dnb['date'],
                y=dfl_dnb['DNB_mean'],
                mode='markers',
                marker=dict(size=6, color='blue'),
                name='Observed',
                error_y=dict(
                    type='data',
                    array=dfl_dnb['DNB_stdDev'],
                    visible=True,
                    color='blue',
                    thickness=0.5,
                    width=2  # width of error bar cap
                ),
                showlegend=True,
            ),
            row=2, col=1
        )
        trace_settings.append((extent, loc, 'dnb'))

# --- Helper for visibility control ---
def vis_array(sel_extent, sel_loc):
    return [
        (extent == sel_extent and loc == sel_loc)
        for extent, loc, _ in trace_settings
    ]

# --- Dropdowns/buttons for location and extent ---
location_buttons = [
    dict(
        label=loc, method='update',
        args=[
            {'visible': vis_array('3x3', loc)},
            {'title': f'NTL Radiance – 3 px radius – {loc}'}
        ]
    )
    for loc in locations
]
extent_buttons = [
    dict(
        label=extent_label_map[extent], method='update',
        args=[
            {'visible': vis_array(extent, locations[0])},
            {'title': f'NTL Radiance – {extent_label_map[extent]} – {locations[0]}'}
        ]
    )
    for extent in extent_codes
]

# --- Set initial visibility: 3x3, Tacloban by default
initial_visible = vis_array('3x3', locations[0])
for i, vis in enumerate(initial_visible):
    fig.data[i].visible = vis

# --- Haiyan Landfall line/annotation
fig.add_vline(
    x=pd.to_datetime("2013-11-08"),
    line_width=2, line_dash="dash", line_color="blue"
)
fig.add_annotation(
    x="2013-11-08", y=1.02, yref='paper',
    text="Haiyan Landfall", showarrow=False,
    font=dict(color="black"), bgcolor="white",
    bordercolor="black", borderwidth=1, xanchor="left"
)

# --- Layout ---
fig.update_layout(
    updatemenus=[
        dict(type="dropdown", direction="down", buttons=location_buttons,
             showactive=True, x=1.1, xanchor="center", y=0.6, yanchor="bottom"),
        dict(type="buttons", direction="right", buttons=extent_buttons,
             showactive=True, x=1.0, y=1.15, xanchor="right", yanchor="top")
    ],
    height=700, width=1200, template='plotly_white',
    paper_bgcolor='rgba(0,0,0,0)',
    xaxis2=dict(title='Date', rangeslider=dict(visible=True)),
    yaxis=dict(title='Px Coverage (%)', range=[0, 100]),
    yaxis2=dict(title='Radiance (nW·cm⁻²·sr⁻¹)')#, range=[0, 2])
)

fig.show()

In [17]:
import pandas as pd
import plotly.express as px

# --- Assume your DataFrame is named df and filtered for 5x5 patch ---
df5 = df[df['extent'] == '5x5'].copy()
df5['date'] = pd.to_datetime(df5['date'])
df5 = df5.sort_values(['location', 'date'])

# Pivot for heatmap
heatmap_data = df5.pivot(index='location', columns='date', values='Valid_pct')

# Custom colorscale: 0 = fully transparent, then light to dark green
custom_colorscale = [
    [0.0, "rgba(0,0,0,0)"],         # 0%: fully transparent
    [0.01, "rgba(204,255,204,0.2)"],# very low (pale green, slight alpha)
    [0.25, "rgba(102,255,102,0.7)"],# light green
    [0.5, "rgba(34,177,76,0.85)"],  # medium green
    [0.75, "rgba(0,100,0,1)"],      # dark green
    [1.0, "rgba(0,60,0,1)"]         # full dark green
]

fig = px.imshow(
    heatmap_data,
    color_continuous_scale=custom_colorscale,
    aspect='auto',
    labels=dict(x="Date", y="Location", color="Coverage (%)"),
    title="Observed 5x5 Pixel Coverage (%)"
)

fig.update_layout(
    height=200 + 20 * len(heatmap_data),
    width=1200,
    xaxis=dict(side="bottom"),
    paper_bgcolor='rgba(0,0,0,0)',
    yaxis=dict(title="Location"),
    coloraxis_colorbar=dict(title="Coverage (%)")
)

# Haiyan landfall annotation
fig.add_vline(
    x=pd.to_datetime("2013-11-08"),
    line_width=2, line_dash="dash", line_color="red"
)
fig.add_annotation(
    x="2013-11-08", y=1.01, yref='paper',
    text="Haiyan Landfall", showarrow=False,
    font=dict(color="black"), bgcolor="white",
    bordercolor="black", borderwidth=1, xanchor="left"
)

fig.show()

## III. Summary Table Algorithm for VIIRS Patch Time Series

This summary statistics process generates key indicators for each point of interest (POI) and patch extent (1×1, 3×3, 5×5 pixels) using VIIRS nighttime lights data. The design ensures valid comparison across patch sizes and robustly incorporates data quality flags.

---

### **Key Steps**

1. **Patch Extent Definition**
    - `1x1` patch: 1 pixel per day
    - `3x3` patch: 9 pixels per day
    - `5x5` patch: 25 pixels per day

2. **Valid and Gap-Filled Pixel Counting**
    - For each POI and extent, count the total number of valid (fresh, HQ=0) and gap-filled pixels across all days.
    - For `1x1`, this is simply the number of days the pixel is valid or gap-filled.
    - For `3x3` and `5x5`, sum valid/gap-filled pixels across all days (e.g., Valid Pixels = sum of `DNB_valid_px`).

3. **Percent Calculation**
    - **Valid Pixels (%)**:  
      ```
      Valid Pixels (%) = (Total Valid Pixels) / (Number of Days × Pixels per Patch) × 100
      ```
    - **Gap-Filled Pixels (%)**:  
      ```
      Gap-Filled Pixels (%) = 100 - Valid Pixels (%)
      ```
    - Both the count and percent are shown for interpretability (e.g., "115 (68.4%)").

4. **Quality Flags from `HQ_hist`**
    - **% HQ = 0 (Same-day)**:  
      Proportion of all pixels (days × patch size) classified as "fresh" (HQ=0), based on `HQ_hist`.
    - **Median Latest HQ Retrieval (days)**:  
      Median value from all `HQ_hist` retrievals, showing the typical "age" of data used for each pixel.

5. **Mean Radiance and Dropout**
    - **DNB Mean** and **Gap-Filled Mean ± STD** are reported for each patch/POI.
    - **Max Dropout (days)**:  
      The longest streak of consecutive days with zero valid pixels, indicating worst-case observation gap.

6. **Consistency for 1x1**
    - For `1x1`, **Valid Pixels (%)** and **% HQ = 0 (Same-day)** are both calculated from `HQ_hist`, so they always match and represent the number of days the pixel is fresh.

---

### **Table Output Example**

| Location         | Valid Pixels (%) | Gap-Filled Pixels (%) | DNB Mean | Gap-Filled Mean ± STD | Max Dropout (days) | Median Latest HQ Retrieval (days) | % HQ = 0 (Same-day) |
|------------------|-----------------|----------------------|----------|----------------------|--------------------|------------------------------|----------------------|
| Tacloban_City    | 119 (32.6%)     | 246 (67.4%)          | 25.19    | 29.43 ± 0.00         | 11                 | 2                            | 25.82                |
| ...              | ...             | ...                  | ...      | ...                  | ...                | ...                          | ...                  |

---

**Notes:**
- For 3x3 and 5x5 patches, percentages represent total pixel coverage across the time period.
- For 1x1, "pixels" = "days."
- All calculations are robust to missing values.

In [18]:
from ast import literal_eval

def parse_hist(series):
    merged = {}
    all_vals = []
    for val in series.dropna():
        h = literal_eval(val) if isinstance(val, str) else val
        all_vals.extend([int(k)] * int(v) for k, v in h.items())
        for k, v in h.items():
            k = int(k)
            merged[k] = merged.get(k, 0) + float(v)
    total = sum(merged.values()) or 1
    return {
        'rel': {k: v / total for k, v in merged.items()},
        'raw': merged,
        'vals': [k for sublist in all_vals for k in sublist]
    }

def max_dropout(valid_flags):
    max_run = run = 0
    for v in valid_flags:
        if v == 0:
            run += 1
            max_run = max(max_run, run)
        else:
            run = 0
    return max_run

summary_dict = {ec: [] for ec in extent_codes}
poi_locations = [poi['name'] for poi in poi_data]

extent_to_total = {'1x1': 1, '3x3': 9, '5x5': 25}

for extent_code in extent_codes:
    df_e = df[df['extent'] == extent_code].copy()
    n_pix = extent_to_total[extent_code]
    for loc in poi_locations:
        dfl = df_e[df_e['location'] == loc].copy()
        if dfl.empty:
            continue

        row = {'Location': loc}

        if extent_code == '1x1':
            total_days = len(dfl)
            # Use HQ_hist as the definitive source for "fresh" pixels
            if 'HQ_hist' in dfl.columns:
                hq = parse_hist(dfl['HQ_hist'])
                fresh_days = int(hq['raw'].get(0, 0))
                hq_vals = hq['vals']
            else:
                fresh_days = dfl['DNB_mean'].notna().sum()
                hq_vals = []
            valid_pct = (fresh_days / total_days * 100) if total_days else 0
            gapfilled_days = total_days - fresh_days
            gapfilled_pct = 100 - valid_pct

            dnb_mean = dfl['DNB_mean'].mean()
            gap_mean = dfl['Gap_mean'].mean()
            gap_std = dfl['Gap_stdDev'].mean()

            row.update({
                'Valid Pixels (%)': f"{fresh_days} ({valid_pct:.1f}%)",
                'Gap-Filled Pixels (%)': f"{gapfilled_days} ({gapfilled_pct:.1f}%)",
                'DNB Mean': f"{dnb_mean:.2f}" if not np.isnan(dnb_mean) else "—",
                'Gap-Filled Mean ± STD': f"{gap_mean:.2f} ± {gap_std:.2f}" if not np.isnan(gap_mean) else "—",
                'Max Dropout (days)': max_dropout((dfl['DNB_mean'].notna()).astype(int)),
                'Median Latest HQ Retrieval (days)': int(np.median(hq_vals)) if hq_vals else '—',
                '% HQ = 0 (Same-day)': round(valid_pct, 2)
            })

        else:
            total_px = len(dfl) * n_pix
            valid_pix = dfl['DNB_valid_px'].fillna(0).sum()
            gapfilled_pix = total_px - valid_pix

            valid_pct = (valid_pix / total_px * 100) if total_px > 0 else 0
            gapfilled_pct = (gapfilled_pix / total_px * 100) if total_px > 0 else 0

            dnb_mean = dfl['DNB_mean'].mean()
            dnb_std = dfl['DNB_stdDev'].mean()
            gap_mean = dfl['Gap_mean'].mean()
            gap_std = dfl['Gap_stdDev'].mean()

            if 'HQ_hist' in dfl.columns:
                hq = parse_hist(dfl['HQ_hist'])
                hq_vals = hq['vals']
                median_hq = int(np.median(hq_vals)) if hq_vals else '—'
                pct_hq0 = round(hq['rel'].get(0, 0) * 100, 2)
            else:
                median_hq = '—'
                pct_hq0 = '—'

            row.update({
                'Valid Pixels (%)': f"{int(valid_pix)} ({valid_pct:.1f}%)",
                'Gap-Filled Pixels (%)': f"{int(gapfilled_pix)} ({gapfilled_pct:.1f}%)",
                'DNB Mean ± STD': f"{dnb_mean:.2f} ± {dnb_std:.2f}" if not np.isnan(dnb_mean) else "—",
                'Gap-Filled Mean ± STD': f"{gap_mean:.2f} ± {gap_std:.2f}" if not np.isnan(gap_mean) else "—",
                'Max Dropout (days)': max_dropout((dfl['DNB_valid_px'].fillna(0) > 0).astype(int)),
                'Median Latest HQ Retrieval (days)': median_hq,
                '% HQ = 0 (Same-day)': pct_hq0
            })

        summary_dict[extent_code].append(row)

summary_columns_1x1 = [
    'Location', 'Valid Pixels (%)', 'Gap-Filled Pixels (%)', 'DNB Mean',
    'Gap-Filled Mean ± STD', 'Max Dropout (days)',
    'Median Latest HQ Retrieval (days)', '% HQ = 0 (Same-day)'
]
summary_columns_patch = [
    'Location', 'Valid Pixels (%)', 'Gap-Filled Pixels (%)', 'DNB Mean ± STD',
    'Gap-Filled Mean ± STD', 'Max Dropout (days)',
    'Median Latest HQ Retrieval (days)', '% HQ = 0 (Same-day)'
]

summary_1x1 = pd.DataFrame(summary_dict['1x1'])[summary_columns_1x1]
summary_3x3 = pd.DataFrame(summary_dict['3x3'])[summary_columns_patch]
summary_5x5 = pd.DataFrame(summary_dict['5x5'])[summary_columns_patch]

In [19]:
summary_1x1

,Location,Valid Pixels (%),Gap-Filled Pixels (%),DNB Mean,Gap-Filled Mean ± STD,Max Dropout (days),Median Latest HQ Retrieval (days),% HQ = 0 (Same-day)
0,Roxas_City,63 (17.3%),302 (82.7%),15.75,23.14 ± 0.00,21,3,17.26
1,Kalibo,63 (17.3%),302 (82.7%),3.62,4.71 ± 0.00,16,3,17.26
2,Iloilo_City,91 (24.9%),274 (75.1%),15.31,17.38 ± 0.00,16,2,24.93
3,Bogo_City,71 (19.5%),294 (80.5%),11.31,13.20 ± 0.00,15,2,19.45
4,Cadiz_City,72 (19.7%),293 (80.3%),9.96,13.51 ± 0.00,29,2,19.73
5,Boracay,67 (18.4%),298 (81.6%),10.83,12.29 ± 0.00,15,3,18.36
6,Estancia,73 (20.0%),292 (80.0%),2.99,4.52 ± 0.00,19,3,20.00
7,Cebu City,85 (23.3%),280 (76.7%),57.99,63.59 ± 0.00,15,2,23.29
8,Bacolod City,84 (23.0%),281 (77.0%),27.01,27.90 ± 0.00,21,2,23.01
9,Puerto Princessa City,87 (23.8%),278 (76.2%),25.48,30.61 ± 0.00,19,3,23.84


In [13]:
summary_3x3

,Location,Valid Pixels (%),Gap-Filled Pixels (%),DNB Mean ± STD,Gap-Filled Mean ± STD,Max Dropout (days),Median Latest HQ Retrieval (days),% HQ = 0 (Same-day)
0,Roxas_City,819 (24.9%),2466 (75.1%),9.91 ± 4.89,14.65 ± 7.03,21,3,17.98
1,Kalibo,776 (23.6%),2509 (76.4%),3.92 ± 3.19,4.83 ± 4.42,16,3,18.65
2,Iloilo_City,876 (26.7%),2409 (73.3%),15.14 ± 6.35,15.74 ± 6.22,16,2,24.85
3,Bogo_City,792 (24.1%),2493 (75.9%),8.09 ± 4.88,9.58 ± 4.67,15,2,19.09
4,Cadiz_City,720 (21.9%),2565 (78.1%),7.32 ± 3.24,9.48 ± 4.37,29,2,20.30
5,Boracay,871 (26.5%),2414 (73.5%),7.29 ± 3.77,8.75 ± 4.33,15,3,17.61
6,Estancia,825 (25.1%),2460 (74.9%),2.17 ± 1.02,2.83 ± 1.45,19,2,22.78
7,Cebu City,886 (27.0%),2399 (73.0%),51.26 ± 12.13,55.03 ± 11.23,15,2,25.00
8,Bacolod City,839 (25.5%),2446 (74.5%),28.25 ± 8.46,30.19 ± 9.33,21,3,24.14
9,Puerto Princessa City,856 (26.1%),2429 (73.9%),19.79 ± 8.89,22.02 ± 9.70,19,3,23.26


In [14]:
summary_5x5

,Location,Valid Pixels (%),Gap-Filled Pixels (%),DNB Mean ± STD,Gap-Filled Mean ± STD,Max Dropout (days),Median Latest HQ Retrieval (days),% HQ = 0 (Same-day)
0,Roxas_City,2215 (24.3%),6910 (75.7%),6.64 ± 4.58,8.89 ± 6.63,21,3,18.52
1,Kalibo,2097 (23.0%),7028 (77.0%),3.28 ± 3.16,3.57 ± 3.94,16,3,19.82
2,Iloilo_City,2395 (26.2%),6730 (73.8%),14.46 ± 7.05,15.06 ± 6.61,16,2,25.00
3,Bogo_City,2057 (22.5%),7068 (77.5%),5.29 ± 4.82,6.01 ± 5.20,15,2,20.96
4,Cadiz_City,1767 (19.4%),7358 (80.6%),4.68 ± 3.66,5.48 ± 5.11,29,3,20.14
5,Boracay,2260 (24.8%),6865 (75.2%),5.11 ± 3.85,5.24 ± 4.74,15,3,19.02
6,Estancia,2046 (22.4%),7079 (77.6%),1.48 ± 1.07,1.44 ± 1.50,19,2,24.31
7,Cebu City,2397 (26.3%),6728 (73.7%),47.28 ± 13.01,50.23 ± 12.34,15,2,24.95
8,Bacolod City,2229 (24.4%),6896 (75.6%),22.71 ± 9.15,24.11 ± 8.82,21,2,24.64
9,Puerto Princessa City,2273 (24.9%),6852 (75.1%),12.18 ± 9.73,13.02 ± 10.16,19,3,22.48
